# Middleware Composition with LangChain Agents

This notebook demonstrates how to compose multiple Redis middleware together and use them with LangChain agents using the standard `create_agent` pattern.

## Key Features

- **Combine multiple middleware**: Stack caching, memory, and routing
- **MiddlewareStack**: Compose middleware into a single unit
- **Connection sharing**: Share Redis connections with checkpointers
- **Factory functions**: Quick setup with `create_caching_stack`

## Prerequisites

- Redis 8.0+ or Redis Stack
- OpenAI API key

## Note on Async Usage

The Redis middleware uses async methods internally. When using with `create_agent`, you must use `await agent.ainvoke()` rather than `agent.invoke()`.

## Setup

Install required packages and set API keys.

In [1]:
%%capture --no-stderr
%pip install -U langgraph-checkpoint-redis langchain langchain-openai sentence-transformers

In [2]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")

REDIS_URL = os.environ.get("REDIS_URL", "redis://redis:6379")

OPENAI_API_KEY:  ········


## Using Multiple Middleware with create_agent

You can pass multiple middleware directly to `create_agent`. They are applied in order.

In [ ]:
import ast
import operator as op
import time

from langchain_core.tools import tool

# Safe math evaluator - no arbitrary code execution
SAFE_OPS = {
    ast.Add: op.add, ast.Sub: op.sub, ast.Mult: op.mul,
    ast.Div: op.truediv, ast.Pow: op.pow, ast.USub: op.neg,
}

def _eval_node(node):
    if isinstance(node, ast.Constant):
        return node.value
    elif isinstance(node, ast.BinOp) and type(node.op) in SAFE_OPS:
        return SAFE_OPS[type(node.op)](_eval_node(node.left), _eval_node(node.right))
    elif isinstance(node, ast.UnaryOp) and type(node.op) in SAFE_OPS:
        return SAFE_OPS[type(node.op)](_eval_node(node.operand))
    raise ValueError("Unsupported expression")

def safe_eval(expr: str) -> float:
    return _eval_node(ast.parse(expr, mode='eval').body)

# Track executions
tool_calls = {"search": 0, "calculate": 0}


@tool
def search(query: str) -> str:
    """Search the web for information."""
    tool_calls["search"] += 1
    time.sleep(0.5)  # Simulate API call
    return f"Search results for '{query}': Found relevant information."


@tool
def calculate(expression: str) -> str:
    """Evaluate a mathematical expression."""
    tool_calls["calculate"] += 1
    try:
        result = safe_eval(expression)
        return f"{expression} = {result}"
    except Exception as e:
        return f"Error: {str(e)}"


tools = [search, calculate]
print("Tools defined: search, calculate")

In [4]:
from langchain.agents import create_agent

from langgraph.middleware.redis import (
    SemanticCacheConfig,
    SemanticCacheMiddleware,
    ToolCacheConfig,
    ToolResultCacheMiddleware,
)

# Define which tools are deterministic (same input = same output)
DETERMINISTIC_TOOLS = ["search", "calculate"]

# Create semantic cache for LLM responses
# Note: deterministic_tools should match the cacheable tools
semantic_cache = SemanticCacheMiddleware(
    SemanticCacheConfig(
        redis_url=REDIS_URL,
        name="composition_llm_cache",
        ttl_seconds=3600,
        # When tool results are from these tools, caching is still OK
        deterministic_tools=DETERMINISTIC_TOOLS,
    )
)

# Create tool cache for tool results
# Tools can also use metadata={"cacheable": True/False} on the tool itself
tool_cache = ToolResultCacheMiddleware(
    ToolCacheConfig(
        redis_url=REDIS_URL,
        name="composition_tool_cache",
        # Fallback when tool.metadata["cacheable"] is not set
        cacheable_tools=DETERMINISTIC_TOOLS,
        ttl_seconds=1800,
    )
)

print("Created coordinated middleware:")
print(f"- Deterministic tools: {DETERMINISTIC_TOOLS}")
print("- SemanticCacheMiddleware: caches LLM responses")
print("- ToolResultCacheMiddleware: caches tool results")
print("\nBoth middlewares are aware of which tools are safe to cache!")

Created coordinated middleware:
- Deterministic tools: ['search', 'calculate']
- SemanticCacheMiddleware: caches LLM responses
- ToolResultCacheMiddleware: caches tool results

Both middlewares are aware of which tools are safe to cache!


In [5]:
# Create agent with multiple middleware
agent = create_agent(
    model="gpt-4o-mini",
    tools=tools,
    middleware=[semantic_cache, tool_cache],  # Multiple middleware!
)

print("Agent created with both SemanticCache and ToolCache middleware!")

Agent created with both SemanticCache and ToolCache middleware!


In [6]:
from langchain_core.messages import HumanMessage

# Reset counters
tool_calls = {"search": 0, "calculate": 0}

print("Test 1: Search query")
print("=" * 50)
result1 = await agent.ainvoke({"messages": [HumanMessage(content="Search for Python tutorials")]})
print(f"Response: {result1['messages'][-1].content[:100]}...")
print(f"Tool calls: {tool_calls}")

print("\nTest 2: Similar search query (should hit cache)")
print("=" * 50)
result2 = await agent.ainvoke({"messages": [HumanMessage(content="Find Python tutorials online")]})
print(f"Response: {result2['messages'][-1].content[:100]}...")
print(f"Tool calls: {tool_calls}")
print("Note: tool_calls should not increase if cache hit!")

Test 1: Search query


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/205 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

This vectorizer has no async embed method. Falling back to sync.
This vectorizer has no async embed method. Falling back to sync.
This vectorizer has no async embed method. Falling back to sync.
This vectorizer has no async embed method. Falling back to sync.


Response: I found some relevant information about Python tutorials. Here are a few types of tutorials you migh...
Tool calls: {'search': 1, 'calculate': 0}

Test 2: Similar search query (should hit cache)
Response: I found some relevant information about Python tutorials. Here are a few types of tutorials you migh...
Tool calls: {'search': 1, 'calculate': 0}
Note: tool_calls should not increase if cache hit!


## Using MiddlewareStack

The `MiddlewareStack` class lets you compose multiple middleware into a single unit that can be passed to `create_agent`.

In [7]:
from langgraph.middleware.redis import (
    ConversationMemoryConfig,
    ConversationMemoryMiddleware,
    MiddlewareStack,
)

# Create a stack with cache + memory
stack = MiddlewareStack(
    [
        SemanticCacheMiddleware(
            SemanticCacheConfig(
                redis_url=REDIS_URL,
                name="stack_llm_cache",
                ttl_seconds=3600,
            )
        ),
        ConversationMemoryMiddleware(
            ConversationMemoryConfig(
                redis_url=REDIS_URL,
                name="stack_memory",
                session_tag="stack_demo",
                top_k=3,
            )
        ),
    ]
)

print("MiddlewareStack created with:")
print("- SemanticCacheMiddleware")
print("- ConversationMemoryMiddleware")

MiddlewareStack created with:
- SemanticCacheMiddleware
- ConversationMemoryMiddleware


In [8]:
# Create agent with the stack (stack is also an AgentMiddleware!)
agent_with_stack = create_agent(
    model="gpt-4o-mini",
    tools=tools,
    middleware=[stack],  # Pass the stack as a single middleware
)

print("Agent created with MiddlewareStack!")

# Test it
result = await agent_with_stack.ainvoke({"messages": [HumanMessage(content="Hi, I'm testing the middleware stack!")]})
print(f"Response: {result['messages'][-1].content}")

Agent created with MiddlewareStack!


This vectorizer has no async embed method. Falling back to sync.
This vectorizer has no async embed method. Falling back to sync.


Response: Hello! It sounds like you're working on something interesting. How can I assist you with your testing?


## Factory Functions

For common patterns, use factory functions like `create_caching_stack` and `from_configs`.

In [9]:
from langgraph.middleware.redis import create_caching_stack

# Quick setup for caching both LLM and tool results
caching_stack = create_caching_stack(
    redis_url=REDIS_URL,
    semantic_cache_name="factory_llm_cache",
    semantic_cache_ttl=3600,
    tool_cache_name="factory_tool_cache",
    tool_cache_ttl=1800,
    cacheable_tools=["search", "calculate"],
)

print("Created caching stack with create_caching_stack()")
print(f"Number of middleware in stack: {len(caching_stack._middlewares)}")

Created caching stack with create_caching_stack()
Number of middleware in stack: 2


In [10]:
from langgraph.middleware.redis import from_configs

# Create stack from config objects
custom_stack = from_configs(
    configs=[
        SemanticCacheConfig(
            name="custom_llm_cache",
            distance_threshold=0.15,
            ttl_seconds=3600,
        ),
        ToolCacheConfig(
            name="custom_tool_cache",
            cacheable_tools=["search"],
            excluded_tools=["calculate"],
            ttl_seconds=600,
        ),
        ConversationMemoryConfig(
            name="custom_memory",
            session_tag="custom_session",
            top_k=5,
        ),
    ],
    redis_url=REDIS_URL,
)

print("Created custom stack with from_configs()")
print(f"Number of middleware in stack: {len(custom_stack._middlewares)}")

Created custom stack with from_configs()
Number of middleware in stack: 3


## Connection Sharing with Checkpointer

Use `IntegratedRedisMiddleware` to share Redis connections with checkpointers for production deployments.

**Note**: When using async agent methods (`ainvoke`), you must use `AsyncRedisSaver` instead of the sync `RedisSaver`.

In [11]:
from langgraph.checkpoint.redis.aio import AsyncRedisSaver
from langgraph.middleware.redis import IntegratedRedisMiddleware

# Use AsyncRedisSaver for async operations
async_checkpointer = AsyncRedisSaver(redis_url=REDIS_URL)
await async_checkpointer.asetup()

# Create middleware stack that shares connection
integrated_stack = IntegratedRedisMiddleware.from_saver(
    async_checkpointer,
    configs=[
        SemanticCacheConfig(name="integrated_cache", ttl_seconds=3600),
        ToolCacheConfig(name="integrated_tools", ttl_seconds=1800),
    ],
)

print("Created IntegratedRedisMiddleware from AsyncRedisSaver!")
print(f"Number of middleware: {len(integrated_stack._middlewares)}")

# Create agent with both checkpointer and middleware
integrated_agent = create_agent(
    model="gpt-4o-mini",
    tools=tools,
    checkpointer=async_checkpointer,
    middleware=[integrated_stack],
)

# Test it
result = await integrated_agent.ainvoke(
    {"messages": [HumanMessage(content="Hello!")]}, config={"configurable": {"thread_id": "integrated-test"}}
)
print(f"\nResponse: {result['messages'][-1].content}")

Created IntegratedRedisMiddleware from AsyncRedisSaver!
Number of middleware: 2


This vectorizer has no async embed method. Falling back to sync.
This vectorizer has no async embed method. Falling back to sync.



Response: Hello! How can I assist you today?


## Summary

- **Multiple middleware**: Pass a list to `create_agent(middleware=[...])`
- **MiddlewareStack**: Compose middleware into a single unit
- **create_caching_stack()**: Quick setup for LLM + tool caching
- **from_configs()**: Create stack from config objects
- **IntegratedRedisMiddleware**: Share connections with checkpointers

## Cleanup

In [12]:
# Close all middleware
await semantic_cache.aclose()
await tool_cache.aclose()
await stack.aclose()
await caching_stack.aclose()
await custom_stack.aclose()

# Close the async checkpointer
try:
    await async_checkpointer.aclose()
except Exception:
    pass

print("All middleware closed.")
print("Demo complete!")

All middleware closed.
Demo complete!
